<a href="https://colab.research.google.com/github/tomonari-masada/course2023-sml/blob/main/07_linear_regression_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 重回帰による住宅価格の予測

* California housing datasetという有名なデータセットを使う。

 * scikit-learnからロードできるバージョンは、前処理が済んだキレイなデータなので、ここでは使わない。

* データの取得や前処理の一部は、
[Aurélien Géron. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition.](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/) の2章と同じ。

* 機械学習において線形回帰モデルを使うときは、予測性能が最重要
 * 予測性能が上がるなら何でもする、という考え方。
* データ集合について理解を深めるために線形回帰を場合、話はまた別。

In [ ]:
import numpy as np
from scipy import stats, special
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

%config InlineBackend.figure_format = 'retina'

## 1) データを取得

In [ ]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
  os.makedirs(housing_path, exist_ok=True)
  tgz_path = os.path.join(housing_path, "housing.tgz")
  urllib.request.urlretrieve(housing_url, tgz_path)
  housing_tgz = tarfile.open(tgz_path)
  housing_tgz.extractall(path=housing_path)
  housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
def load_housing_data(housing_path=HOUSING_PATH):
  csv_path = os.path.join(housing_path, "housing.csv")
  return pd.read_csv(csv_path)

（ここより上の詳細はフォローしなくてもいいいです。）

In [ ]:
housing = load_housing_data()
housing.head()

## 2) データを概観しつつ前処理

In [ ]:
housing.info()

* 数値データではない列が一つだけある

In [ ]:
housing['ocean_proximity'].value_counts()

* この数値データではない列を消してしまう
 * ここを変更してもいいです。

In [ ]:
housing_num = housing.drop('ocean_proximity', axis=1)

In [ ]:
housing_num.info()

* total_bedroomsは欠測箇所がある。
 * 後で対処する。

* median_house_valueを予測する、という問題設定。
 * これ以外は特徴量として使う。

In [ ]:
X = housing_num.drop('median_house_value', axis=1)
y = housing_num["median_house_value"].copy()

## 3) 評価実験のための準備

* 訓練データ、検証データ、テストデータに分ける
 * 今回は、6:2:2になるように分ける。（この比率に深い意味はない。）

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
print(X_train.shape, X_valid.shape, X_test.shape)

* total_bedroomsの値が欠けているエントリがある
  * ここでは単に削除することにする（ここを変更してもいいです）。
  * isna()メソッドでNA valuesがある行を調べて、Xとyの両方に使う。

In [ ]:
na_index = X_train.isna().any(axis=1)
X_train = X_train[~ na_index]
y_train = y_train[~ na_index]

na_index = X_valid.isna().any(axis=1)
X_valid = X_valid[~ na_index]
y_valid = y_valid[~ na_index]

na_index = X_test.isna().any(axis=1)
X_test = X_test[~ na_index]
y_test = y_test[~ na_index]

In [ ]:
print(X_train.shape, X_valid.shape, X_test.shape)

* 交差検証をしてもらってもいいです。

## 4) 訓練データをよくよくながめてみる

 * EDA (exploratory data analysis) をおこなう

In [ ]:
X_train.describe()

### ヒストグラム
 * cf. Aurélien Géron. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow, 2nd Edition. p.50

In [ ]:
X_train.hist(bins=50, figsize=(15,12));

* total_roomsの値の分布に注目してみる

In [ ]:
plt.hist(X_train.total_rooms, bins=50);

* 説明変数の値がどのように分布するかは、回帰モデルの予測性能に直接は関係しない。
 * 回帰モデルでは、誤差項が正規分布に従うという仮定はすることがある（検定を行う場合）。
 * しかし、説明変数の値の分布については、何も仮定しない。
  * 例えば、0か1かの2通りの値しかとらない説明変数もよく使う。
 * とはいえ、それで予測性能が上がるなら、説明変数の値の分布を変更してみる余地はある。

* total_roomsについて、scipyのBox-Cox変換を適用して、ヒストグラムを描いてみる
 * scikit-learnでもBox-Cox変換はできる。

In [ ]:
boxcox_, maxlog_ = stats.boxcox(X_train.total_rooms)
plt.hist(boxcox_, bins=50);
print(f"maxlog_  for train total_rooms: {maxlog_:.4f}")
# （maxlog_ の部分は、test setなど、別のデータ集合を同じ条件で変換するときに使う。）

* Box-Cox変換の戻し方

In [ ]:
plt.hist(special.inv_boxcox(boxcox_, maxlog_), bins=50);

* 変換した後のデータを使うほうが予測性能がよくなるかどうか、後で試してみよう。

* housing_median_ageを見てみる
 * 最大値の頻度が妙に高い。

In [ ]:
X_train.housing_median_age.hist(bins=50);

In [ ]:
X_train.housing_median_age.value_counts().head()

### 相関係数のヒートマップ
 * 多重共線性 https://bellcurve.jp/statistics/glossary/1792.html

In [ ]:
plt.subplots(figsize=(10,10))
sns.heatmap(X_train.corr(), annot=True, square=True);

### ペア・プロット

In [ ]:
sns.pairplot(X_train, diag_kind='kde');

## 5) 線形回帰

* training set上でモデル・パラメータを決定する


In [ ]:
reg = LinearRegression()
reg.fit(X_train, y_train)

* validation set上で評価する


In [ ]:
y_valid_pred = reg.predict(X_valid)
rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
print(f'RMSE: {rmse:f}')

* ここでターゲット（目的変数）の値の分布を見てみる

In [ ]:
y_train.hist(bins=50);

In [ ]:
y_train.value_counts().head()

* 予測値が、訓練データ内でのターゲットの最大値を超えないようにして、再び評価する

In [ ]:
y_valid_pred = reg.predict(X_valid)
y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
print(f'RMSE: {rmse:f}')

## 6) リッジ回帰
* 係数の二乗の和を抑える正則化を含む。

In [ ]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=1.0)
reg.fit(X_train, y_train)
y_valid_pred = reg.predict(X_valid)
y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
print('RMSE: {:f}'.format(mean_squared_error(y_valid, y_valid_pred, squared=False)))

## 7) Lasso
* 係数の絶対値の和を抑える正則化を含む。

In [ ]:
from sklearn.linear_model import Lasso

reg = Lasso(alpha=1.0)
reg.fit(X_train, y_train)
y_valid_pred = reg.predict(X_valid)
y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
print('RMSE: {:f}'.format(mean_squared_error(y_valid, y_valid_pred, squared=False)))

## 8) 試行錯誤するための選択肢

### 説明変数の値を加工
 * Box-Cox変換（既述）
 * MinMaxScaler

* スケーリングは訓練データのみで行うほうが良い
 * k-近傍法の2つ目の課題では、テストデータ以外のデータ全てを使ってスケーリングしていた。訓練データのみで（つまり、検証データの一国を除いて）スケーリングするとどうなるか、時間があるときに試してみよう。

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train) # スケーラのfitは訓練データのみで行う
X_train_scaled = X_train.copy()
X_train_scaled[X_train.columns] = scaler.transform(X_train)
X_valid_scaled = X_valid.copy()
X_valid_scaled[X_valid.columns] = scaler.transform(X_valid) # 検証データに同じスケーリングを適用

In [ ]:
X_train_scaled.describe()

### 正則化パラメータをチューニング

In [ ]:
for alpha in 10. ** np.arange(-6, 7):
  reg = Ridge(alpha=alpha, random_state=123)
  reg.fit(X_train_scaled, y_train)
  y_valid_pred = reg.predict(X_valid_scaled)
  y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
  rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
  print(f'alpha: {alpha:f}; RMSE: {rmse:f}')

In [ ]:
reg = LinearRegression()
reg.fit(X_train_scaled, y_train)
y_valid_pred = reg.predict(X_valid_scaled)
y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
print(f'RMSE: {rmse:f}')

### 試行錯誤の例：新しい説明変数を作成

* 下の例では、何をしているだろうか？

In [ ]:
print(X_train.longitude.median(), X_train.latitude.median())

In [ ]:
med_lon = X_train.longitude.median()
med_lat = X_train.latitude.median()

In [ ]:
flag_lon = (X_train.longitude < med_lon) * 1
flag_lat = (X_train.latitude < med_lat) * 1

X_train['lon'] = flag_lon
X_train['lat'] = flag_lat

In [ ]:
X_train.describe()

In [ ]:
flag_lon = (X_valid.longitude < med_lon) * 1
flag_lat = (X_valid.latitude < med_lat) * 1

X_valid['lon'] = flag_lon
X_valid['lat'] = flag_lat

In [ ]:
scaler = MinMaxScaler()
scaler.fit(X_train) # スケーラのfitは訓練データのみで行う
X_train_scaled = X_train.copy()
X_train_scaled[X_train.columns] = scaler.transform(X_train)
X_valid_scaled = X_valid.copy()
X_valid_scaled[X_valid.columns] = scaler.transform(X_valid) # 検証データに同じスケーリングを適用

In [ ]:
reg = LinearRegression()
reg.fit(X_train_scaled, y_train)
y_valid_pred = reg.predict(X_valid_scaled)
y_valid_pred[y_valid_pred > y_train.max()] = y_train.max()
rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
print(f'RMSE: {rmse:f}')

In [ ]:
plt.plot(y_valid_pred, np.abs(y_valid_pred - y_valid), '.');

# 課題7

* RMSEによって評価される予測性能を、良くして下さい
* test setとそれ以外の部分の分割は、変えないでください
 * test set以外の部分をどう使うかは、自由です。
 * training setとvalidation setをくっつけて、交差検証をしていいです。
* リッジ回帰とLassoを使ってもいいです
* 高次多項式特徴量を使ってもいいです（cf. `sklearn.preprocessing.PolynomialFeatures`）
* test setでのRMSEによる評価は最後に一回おこなうだけです